In [3]:
# To ensure Chinese characters are displayed correctly
options(encoding = "UTF-8")
Sys.setlocale("LC_CTYPE", "zh_TW.UTF-8")

[1] "zh_TW.UTF-8"

In [24]:
flights <- readRDS("data/flights_week10.rds")

In [25]:
flightsData <- flights$data[[1]]$data_frame

In [26]:
dplyr::glimpse(flightsData)

Rows: 4,941
Columns: 24
$ AirlineID          <fct> 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 5J,…
$ ScheduleStartDate  <date> 2023-10-13, 2023-10-20, 2023-10-27, 2023-10-13, 20…
$ ScheduleEndDate    <chr> "2023-10-15", "2023-10-22", "2023-10-27", "2023-10-…
$ FlightNumber       <chr> "3U3783", "3U3783", "3U3783", "3U3784", "3U3784", "…
$ DepartureAirportID <chr> "CKG", "CKG", "CKG", "TSA", "TSA", "TSA", "TFU", "T…
$ DepartureTime      <chr> "2023-10-13 15:00", "2023-10-20 15:00", "2023-10-27…
$ CodeShare          <list> [<data.frame[0 x 0]>], [<data.frame[0 x 0]>], [<da…
$ ArrivalAirportID   <chr> "TSA", "TSA", "TSA", "CKG", "CKG", "CKG", "TSA", "T…
$ ArrivalTime        <chr> "18:00", "18:00", "18:00", "22:15", "22:15", "22:15…
$ Monday             <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, TRUE, TRU…
$ Tuesday            <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ Wednesday          <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, TRUE, TRU…
$ Thursday      

Data frame `flightsData` has a string column `DepartureTime` with values like "2023-10-13 15:00". There is another column `DepartureTimeZone` with values like "Asia/Taipei" telling us the `DepartureTime` time zone for each value. How to convert `DepartureTime` to a `datetime` column with the correct time zone, using dplyr and lubridate packages in R? 

In [8]:

library(dplyr)
library(lubridate)

flightsData <- flightsData %>%
  group_by(DepartureTimeZone) %>%
  mutate(DepartureTime = ymd_hms(DepartureTime, tz = DepartureTimeZone[[1]])) |>
  glimpse()


Rows: 4,941
Columns: 24
Groups: DepartureTimeZone [35]
$ AirlineID          <fct> 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 5J,…
$ ScheduleStartDate  <date> 2023-10-13, 2023-10-20, 2023-10-27, 2023-10-13, 20…
$ ScheduleEndDate    <chr> "2023-10-15", "2023-10-22", "2023-10-27", "2023-10-…
$ FlightNumber       <chr> "3U3783", "3U3783", "3U3783", "3U3784", "3U3784", "…
$ DepartureAirportID <chr> "CKG", "CKG", "CKG", "TSA", "TSA", "TSA", "TFU", "T…
$ DepartureTime      <dttm> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ CodeShare          <list> [<data.frame[0 x 0]>], [<data.frame[0 x 0]>], [<da…
$ ArrivalAirportID   <chr> "TSA", "TSA", "TSA", "CKG", "CKG", "CKG", "TSA", "T…
$ ArrivalTime        <chr> "18:00", "18:00", "18:00", "22:15", "22:15", "22:15…
$ Monday             <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, TRUE, TRU…
$ Tuesday            <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ Wednesday          <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FAL

# Parse time with different time zone

Our data come from different time zone. We can use `lubridate::ymd_hm(..., tz={time zone})` to parse `...` into the same `{time zone}` value. 

## Split data according to time zone

In [9]:
# 3. parse time with time zone
## split flightsData according to its timezone
flightsData |> split(flightsData$DepartureTimeZone) -> split_flightsData

In [15]:
head(split_flightsData[[1]], 3)
head(split_flightsData[[2]]$DepartureTimeZone, 3)
head(split_flightsData[[3]]$DepartureTimeZone, 3)

AirlineID,ScheduleStartDate,ScheduleEndDate,FlightNumber,DepartureAirportID,DepartureTime,CodeShare,ArrivalAirportID,ArrivalTime,Monday,⋯,Saturday,Sunday,UpdateTime,VersionID,Terminal,num_codeShare,DepartureTimeZone,ArrivalTimeZone,DepartTaiwan,ArriveTaiwan
<fct>,<date>,<chr>,<chr>,<chr>,<dttm>,<list>,<chr>,<chr>,<lgl>,⋯,<lgl>,<lgl>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<lgl>,<lgl>
BR,2023-10-09,2023-10-15,BR051,IAH,NA,NULL,TPE,06:00+1,TRUE,⋯,TRUE,TRUE,2023-10-10T08:26:07+08:00,1111,2,0,America/Chicago,Asia/Taipei,FALSE,TRUE
BR,2023-10-16,2023-10-22,BR051,IAH,NA,NULL,TPE,06:00+1,TRUE,⋯,TRUE,TRUE,2023-10-10T08:26:07+08:00,1111,2,0,America/Chicago,Asia/Taipei,FALSE,TRUE
BR,2023-10-23,2023-10-28,BR051,IAH,NA,NULL,TPE,06:00+1,TRUE,⋯,TRUE,FALSE,2023-10-10T08:26:07+08:00,1111,2,0,America/Chicago,Asia/Taipei,FALSE,TRUE


[1] "America/Los_Angeles" "America/Los_Angeles" "America/Los_Angeles"

[1] "America/New_York" "America/New_York" "America/New_York"

## Parsing a time zone


In [16]:
# for each sub data frame
.x=1 # say the first one
# we want to parse the time
split_flightsData[[.x]]$DepartureTime <- 
  lubridate::ymd_hm(split_flightsData[[.x]]$DepartureTime, tz = split_flightsData[[.x]]$DepartureTimeZone[[1]]) 


- `tz = groupXdata$DepartureTimeZone[[1]]` only take ONE time zone value because the same sub data frame has the same time zone. Also `tz=` can take in only one string.


### dplyr::mutate

If you have an expression like

```
data_frame$some_column <- operations on ... other data_frame$column(s)
```

you can use `dplyr::mutate` to do the same thing:

```
data_frame |>
  dplyr::mutate(
    some_column = operations on ... other column(s)
  )
```

In our previous example `data_frame` is `split_flightData[[.x]]`, so we can write

In [18]:
split_flightsData[[.x]] |>
  dplyr::mutate(
    DepartureTime = 
      lubridate::ymd_hm(DepartureTime, tz = DepartureTimeZone[[1]])
  )

AirlineID,ScheduleStartDate,ScheduleEndDate,FlightNumber,DepartureAirportID,DepartureTime,CodeShare,ArrivalAirportID,ArrivalTime,Monday,⋯,Saturday,Sunday,UpdateTime,VersionID,Terminal,num_codeShare,DepartureTimeZone,ArrivalTimeZone,DepartTaiwan,ArriveTaiwan
<fct>,<date>,<chr>,<chr>,<chr>,<dttm>,<list>,<chr>,<chr>,<lgl>,⋯,<lgl>,<lgl>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<lgl>,<lgl>
BR,2023-10-09,2023-10-15,BR051,IAH,NA,NULL,TPE,06:00+1,TRUE,⋯,TRUE,TRUE,2023-10-10T08:26:07+08:00,1111,2,0,America/Chicago,Asia/Taipei,FALSE,TRUE
BR,2023-10-16,2023-10-22,BR051,IAH,NA,NULL,TPE,06:00+1,TRUE,⋯,TRUE,TRUE,2023-10-10T08:26:07+08:00,1111,2,0,America/Chicago,Asia/Taipei,FALSE,TRUE
BR,2023-10-23,2023-10-28,BR051,IAH,NA,NULL,TPE,06:00+1,TRUE,⋯,TRUE,FALSE,2023-10-10T08:26:07+08:00,1111,2,0,America/Chicago,Asia/Taipei,FALSE,TRUE
BR,2023-10-09,2023-10-15,BR055,ORD,NA,NULL,TPE,05:25+1,TRUE,⋯,TRUE,TRUE,2023-10-10T08:26:07+08:00,1111,2,0,America/Chicago,Asia/Taipei,FALSE,TRUE
BR,2023-10-16,2023-10-22,BR055,ORD,NA,NULL,TPE,05:25+1,TRUE,⋯,TRUE,TRUE,2023-10-10T08:26:07+08:00,1111,2,0,America/Chicago,Asia/Taipei,FALSE,TRUE
BR,2023-10-23,2023-10-28,BR055,ORD,NA,NULL,TPE,05:25+1,TRUE,⋯,TRUE,FALSE,2023-10-10T08:26:07+08:00,1111,2,0,America/Chicago,Asia/Taipei,FALSE,TRUE
CM,2023-10-09,2023-10-15,CM8019,ORD,NA,NULL,TPE,05:25+1,TRUE,⋯,TRUE,TRUE,2023-10-10T08:26:07+08:00,1111,2,0,America/Chicago,Asia/Taipei,FALSE,TRUE
CM,2023-10-16,2023-10-22,CM8019,ORD,NA,NULL,TPE,05:25+1,TRUE,⋯,TRUE,TRUE,2023-10-10T08:26:07+08:00,1111,2,0,America/Chicago,Asia/Taipei,FALSE,TRUE
CM,2023-10-23,2023-10-28,CM8019,ORD,NA,NULL,TPE,05:25+1,TRUE,⋯,TRUE,FALSE,2023-10-10T08:26:07+08:00,1111,2,0,America/Chicago,Asia/Taipei,FALSE,TRUE


### dplyr::group_by

So far we have ...

- If you want going to work on every group in your split data frame, which is split based on `DepartureTimeZone`, then you can replace `split_flightsData[[.x]] ` with `flightsData |> dplyr::group_by(DepartureTimeZone) |> dplyr::mutate(...) |> dplyr::ungroup()`


- If we want to `flightsData |> split(flightsData$DepartureTimeZone) ` and then work on each group separately on other dplyr procedure. Then we can use `dplyr::group_by` as
`flightsData |> dplyr::group_by(DepartureTimeZone) |>  {continue to other dplyr procedure}`

In [27]:
flightsData |>
    dplyr::group_by(DepartureTimeZone) |> # split and do the following on each sub data frame
    dplyr::mutate(
        DepartureTime = lubridate::ymd_hm(DepartureTime, tz = DepartureTimeZone[[1]]) # parse time
    ) |>
    dplyr::ungroup() -> # unsplit the data frame
    flightsData2

- end your procedure with `dplyr::ungroup()` to ungroup the data frame.

In [ ]:
dplyr::glimpse(flightsData2)

jupyter nbconvert --to markdown lecture-notes/week10.ipynb
